In this competetition, our goal is to predict pawpularity of pet's profile based on appearance of pet profile. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Loading the .csv file

In [ ]:
tran=pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
tran.head()

Listing train columns name

In [ ]:
tran.columns

Listing test columns names

In [ ]:
test.columns

Filtering columns for x and y variables

In [ ]:
column=[col for col in tran.columns if col not in ['Id','Pawpularity']]

Spliting train and test dataset

In [ ]:
x=tran[column]
y=tran['Pawpularity']

In [ ]:
print(x.shape,y.shape,test.shape)

In [ ]:
test_id=test['Id']
test=test.drop('Id',axis=1)
test.shape

Checking any null instances in train and test dataset

In [ ]:
tran.isnull().any()

In [ ]:
test.isnull().any()

Droping train id column

In [ ]:
tran.drop('Id',axis=1,inplace=True)

**Heatmap analysis**

In [ ]:
tran_matrix=tran.corr()
plt.figure(figsize=(10,8))
sns.heatmap(tran_matrix,linewidth=0.1,linecolor='white',annot=True,cmap='Pastel1',fmt='.2f')
plt.show()

In [ ]:
feature_variables=x.columns
target_variable='Pawpularity'

sns.countplot(x=feature_variables,y=target_variable,data=tran)

Using PCA to reduce dimension for analysis outlier

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2,random_state=42)
pca=pca.fit_transform(x)

Visualization of PCA 

In [ ]:
plt.figure(figsize=(8,7))
plt.scatter(pca[:,0],pca[:,1],color='red')
plt.show()

Checking pawpularity distribution 

In [ ]:
plt.hist(y,bins=10)
plt.show()

In [ ]:
y_log=np.log1p(y)
plt.hist(y_log,bins=10)
plt.show()

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split,GridSearchCV
import xgboost as xgb

In [ ]:
fold=KFold(n_splits=5,random_state=42,shuffle=True)

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y_log,random_state=42,test_size=0.2)

In [ ]:
print(xtrain.shape,ytrain.shape)

**Parameters for Gradient boosting Regressor algorithm**

In [ ]:
param={'n_estimators' : [400,600,700,800,900],
      'max_depth': [3,5,7,9],
      'learning_rate':[0.01,0.05,0.1,0.03],
      'objective':['reg:squarederror'],
      'booster':['gbtree']}

In [ ]:
xgb_regressor=xgb.XGBRegressor()

In [ ]:
xgb_grid=GridSearchCV(xgb_regressor,param,n_jobs=-1,cv=fold)

In [ ]:
xgb_grid.fit(xtrain,ytrain)

Best parameters of gridsearchcv

In [ ]:
xgb_grid.best_params_

**Using parameters of gridsearchcv for training new xgb regressor**

In [ ]:
xgb=xgb.XGBRegressor(**xgb_grid.best_params_)

In [ ]:
xgb=xgb.fit(xtrain,ytrain)

In [ ]:
y_pred=xgb.predict(xtest)

RMSE value

In [ ]:
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(ytest,y_pred)
rmse=np.sqrt(mse)
rmse

Predicting new instances

In [ ]:
sub=xgb.predict(test)
sub.shape

In [ ]:
sub=np.expm1(sub)

In [ ]:
submission=pd.DataFrame({'Id':test_id,'Pawpularity':sub})

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv',index=False)